In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [2]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.321,),(0.562,))])

In [3]:
train_data=datasets.MNIST(root="./data",transform=transform,download=False,train=True)
test_data=datasets.MNIST(root="./data",transform=transform,download=False,train=False)


In [4]:
train_subset=Subset(train_data,range(200))
test_subset=Subset(test_data,range(20))

In [5]:
train_loader=DataLoader(train_subset,batch_size=10,shuffle=True)
test_loader=DataLoader(test_subset,batch_size=10,shuffle=False)

In [19]:
class SimpleCNN(nn.Module):

    def __init__(self):
        super(SimpleCNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(1,10,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(10,20,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.fc1=nn.Linear(20*7*7,50)
        self.fc2=nn.Linear(50,10)

    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x,dim=1)

model=SimpleCNN()
criterion=nn.CrossEntropyLoss()
    


In [20]:
def train_loop(epochs,optimizer):

    for epoch in range(epochs):
        model.train()
        train_loss=0
        correct_train=0
        total_train=0

        for data,target in train_loader:
            output=model(data)
            loss=criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss+=loss.item()
            predicted=torch.argmax(output.data,dim=1)
            total_train+=output.size(0)
            correct_train+=(predicted==target).sum().item()

        avg_train_loss=train_loss/len(train_loader)
        train_acc=100*correct_train/total_train

        model.eval()
        test_loss=0
        correct_test=0
        total_test=0

        for data,target in train_loader:
            output=model(data)
            loss=criterion(output,target)
            test_loss+=loss.item()
            predicted=torch.argmax(output.data,dim=1)
            total_test+=output.size(0)
            correct_test+=(predicted==target).sum().item()

        avg_test_loss=test_loss/len(test_loader)
        test_acc=100*correct_test/total_test

        print(f'Epoch:{epoch+1},train loss:{avg_train_loss}, test loss:{avg_test_loss}, train acc:{train_acc}, test acc:{test_acc}')
            



In [21]:
optimizer=optim.Adam(model.parameters(),lr=0.001)
train_loop(30,optimizer)

Epoch:1,train loss:2.279746782779694, test loss:21.852947473526, train acc:18.5, test acc:32.5
Epoch:2,train loss:2.0804501593112947, test loss:18.476596176624298, train acc:46.0, test acc:63.5
Epoch:3,train loss:1.5718551278114319, test loss:12.092442870140076, train acc:68.5, test acc:69.5
Epoch:4,train loss:0.986744512617588, test loss:7.206243991851807, train acc:74.0, test acc:84.0
Epoch:5,train loss:0.7009362727403641, test loss:5.392041206359863, train acc:79.5, test acc:85.0
Epoch:6,train loss:0.5548729807138443, test loss:3.849776305258274, train acc:84.5, test acc:91.0
Epoch:7,train loss:0.3847118206322193, test loss:2.886092103086412, train acc:87.0, test acc:93.0
Epoch:8,train loss:0.2920407045632601, test loss:2.2643027417361736, train acc:93.5, test acc:95.0
Epoch:9,train loss:0.24217873616144062, test loss:1.8580034114420414, train acc:94.0, test acc:96.0
Epoch:10,train loss:0.21031878497451545, test loss:1.496006865054369, train acc:95.5, test acc:97.5
Epoch:11,train lo

In [22]:
optimizer=optim.SGD(model.parameters(),lr=0.001)
train_loop(30,optimizer)

Epoch:1,train loss:0.0039350849663605915, test loss:0.03909690148429945, train acc:100.0, test acc:100.0
Epoch:2,train loss:0.003928257728694007, test loss:0.03896967621403746, train acc:100.0, test acc:100.0
Epoch:3,train loss:0.003915033876546658, test loss:0.03885502426419407, train acc:100.0, test acc:100.0
Epoch:4,train loss:0.0038992705463897436, test loss:0.0387500841752626, train acc:100.0, test acc:100.0
Epoch:5,train loss:0.003891157390899025, test loss:0.03865142696304247, train acc:100.0, test acc:100.0
Epoch:6,train loss:0.0038869680400239305, test loss:0.038560402346774936, train acc:100.0, test acc:100.0
Epoch:7,train loss:0.0038732923974748703, test loss:0.03847278206376359, train acc:100.0, test acc:100.0
Epoch:8,train loss:0.003869892648071982, test loss:0.038386854372220114, train acc:100.0, test acc:100.0
Epoch:9,train loss:0.003855298738926649, test loss:0.038306127942632884, train acc:100.0, test acc:100.0
Epoch:10,train loss:0.003848525835201144, test loss:0.0382